In [1]:
#%matplotlib inline
import seWrapper as se
#import matplotlib.pyplot as plt
import numpy as np
reload(se)
import astropy.units as u

/Users/zjennings/anaconda/envs/iraf27/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/zjennings/anaconda/envs/iraf27/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
data = np.loadtxt('bg1_r_uncorrected.csv',delimiter=',')

In [266]:
data[0,:]

array([  2.01971243e+02,   2.04980473e+01,   2.54069000e+01,
         4.69500000e-01,   2.15000000e+00,   1.88000000e-01,
         0.00000000e+00])

In [3]:
catalog = se.seCatalog()

In [4]:
catalog.create_new_catalog_arrs(ra=data[:,0],dec=data[:,1],fil='i',mags=data[:,2],\
                                    merr=data[:,3],fwhm=data[:,4],ellipticity=data[:,5],flags=data[:,6])

In [5]:
fake_stars = se.fakeStarTests('field_1/NGVS_BG_1.R.fits',\
                                catalog,config_file='i.sex',seeing=3.,zpt=30.0,\
                              pix_scale=0.187,write_true_phot=False,gain=39.5)

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


In [6]:
fake_stars.genFakeCatalogs(n_stars=10000,min_mag=20,max_mag=27,n_iter=1,overwrite=True)

In [7]:
fake_stars.fake_cats['0'][0,:] - fake_stars.fake_cats['0'][1,:]

array([  1.33811905,  -0.97140564,  -1.15244737, ..., -75.54844976,
       -74.37154179, -76.93722028])

In [8]:
np.savetxt('bg1_r_fake_stars.csv',fake_stars.fake_cats['0'],delimiter=',',fmt='%10.10f')

In [348]:
data = np.loadtxt('bg1_i_fake_stars.csv',delimiter=',')

In [343]:
data.shape

(2, 10000)

In [329]:
phot_cat = fake_stars.phot
starlist_cat = fake_stars.starlist_cat
recovered_cat = fake_stars.recovered_cat
unrec_cat = fake_stars.unrec_cat

In [244]:
starlist_cat.mags['recovered'].shape

(146620,)

In [336]:
bright_rec = ((starlist_cat.mags['input'] < 22) & (starlist_cat.mags['recovered'] > 80))

In [337]:
np.sum(bright_rec)

66

In [149]:
from pyraf.iraf import artdata

In [152]:
mags= np.random.uniform(low=18,high=22,size=1000)

In [153]:
mags = np.random.uniform(low=22,high=26,size=1000)
x = np.random.uniform(high=fake_stars.header['naxis1'],size=1000)
y = np.random.uniform(high=fake_stars.header['naxis2'],size=1000)

In [338]:
def write_reg(ra,dec,size='2"',color='green',shape='circle',filename='regions.reg'):
    f = open(filename,'w')
    f.write('fk5\n')
    for i,j in zip(ra,dec):
        f.write(shape+' '+str(i)+' '+str(j)+' '+size+' # color='+color+'\n')
    f.close()     

In [339]:
ra = starlist_cat.coords[bright_rec].ra.degree
dec = starlist_cat.coords[bright_rec].dec.degree

In [340]:
write_reg(ra,dec,filename='unrec_bright.reg')

In [341]:
starlist_cat.mags['input']

array([ 99.999,  99.999,  99.999, ...,  99.999,  99.999,  99.999])